In [ ]:
# default_exp annotation.viewer

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# export

import sys
import argparse
import logging
import cv2
import numpy as np
from mlcore.image.opencv_tools import fit_to_max_size
from mlcore.annotation.core import RegionShape
from mlcore.annotation.via_adapter import read_annotations

In [ ]:
# hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# export

logger = logging.getLogger(__name__)

# Annotation Viewer
> Annotation Viewer Notes.

In [ ]:
# export


def show_annotated_images(annotations, max_width=0, max_height=0):
    """
    Show images with corresponding annotations.
    Images are shown one at a time with switching by using the arrow left/right keys.
    `annotations`: The annotations for the images
    `max_width`: The maximum width to scale the image for visibility.
    `max_height`: The maximum height to scale the image for visibility.
    """
    len_annotations = len(annotations)

    if len_annotations == 0:
        logger.error("No Annotations found")
        return

    index = 0
    annotation_keys = list(annotations.keys())

    logger.info("Keys to use:")
    logger.info("n = Next Image")
    logger.info("b = Previous Image")
    logger.info("q = Quit")

    logger.info("Annotations to view: {}".format(len_annotations))

    while True:
        annotation_id = annotation_keys[index]
        annotation = annotations[annotation_id]
        logger.info("View Image {}/{}: {}".format(index + 1, len_annotations, annotation.file_path))
        img = cv2.imread(annotation.file_path, cv2.IMREAD_COLOR)

        if img is None:
            logger.info("Image not found at {}".format(annotation.file_path))
            img = np.zeros(shape=(1, 1, 3))

        if annotation.regions:
            logger.info("Found {} regions".format(len(annotation.regions)))
            for region_index, region in enumerate(annotation.regions):
                points = list(zip(region.points_x, region.points_y))
                logger.info("Found {} of category {} with {} points: {}".format(region.shape,
                                                                                ','.join(region.labels),
                                                                                len(points), points))
                if region.shape == RegionShape.CIRCLE:
                    img = cv2.circle(img, points[0], region.radius_x, (0, 255, 255), 2)
                elif region.shape == RegionShape.ELLIPSE:
                    img = cv2.ellipse(img, points[0], (region.radius_x, region.radius_y), 0, 0, 360,
                                      (0, 255, 255), 2)
                elif region.shape == RegionShape.POINT:
                    img = cv2.circle(img, points[0], 1, (0, 255, 255), 2)
                elif region.shape == RegionShape.POLYGON:
                    pts = np.array(points, np.int32)
                    pts = pts.reshape((-1, 1, 2))
                    img = cv2.polylines(img, [pts], True, (0, 255, 255), 2)
                elif region.shape == RegionShape.RECTANGLE:
                    img = cv2.rectangle(img, points[0], points[1], (0, 255, 255), 2)

        if max_width and max_height:
            img = fit_to_max_size(img, max_width, max_height)

        cv2.imshow('Image', img)
        k = cv2.waitKey(0)
        if k == ord('q'):    # 'q' key to stop
            break
        elif k == ord('b'):
            index = max(0, index - 1)
        elif k == ord('n'):
            index = min(len_annotations - 1, index + 1)

    cv2.destroyAllWindows()

## Helper Methods

In [ ]:
# export


def configure_logging(logging_level=logging.INFO):
    """
    Configures logging for the system.

    :param logging_level: The logging level to use.
    """
    logger.setLevel(logging_level)

    handler = logging.StreamHandler(sys.stdout)
    handler.setLevel(logging_level)

    logger.addHandler(handler)

## Run from command line

To run the data-set builder from command line, use the following command:
`python -m mlcore.via.annotation_viewer [parameters]`

The following parameters are supported:
- `path`: The path to the images to view. (e.g.: *imagesets/segmentation/car_damage/trainval*)
- `--annotation`: The path to the VIA annotation file (e.g.: *imagesets/segmentation/car_damage/via_region_data.json*)
- `--category-label-key`: The category label key used, default to `category`

In [ ]:
# export


if __name__ == '__main__' and '__file__' in globals():
    # for direct shell execution
    configure_logging()

    parser = argparse.ArgumentParser()
    parser.add_argument("path",
                        help="The path to the images to view.")
    parser.add_argument("-a",
                        "--annotation",
                        help="The path to the VIA annotation file.")
    parser.add_argument("--max-width",
                        help="The maximum width to scale the image for visibility.",
                        type=int,
                        default=0)
    parser.add_argument("--max-height",
                        help="The maximum height to scale the image for visibility.",
                        type=int,
                        default=0)

    args = parser.parse_args()

    annotations = read_annotations(args.annotation, args.path)
    show_annotated_images(annotations, args.max_width, args.max_height)
